In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_excel("Bank_Personal_Loan_Modelling.xlsx")

df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
# Creating SQLite database
conn = sqlite3.connect("credit_risk.db")

# Loading dataframe into SQL table
df.to_sql("loan_customers", conn, if_exists="replace", index=False)

5000

In [4]:
#Overall loan acceptance rate
query_1 = """
SELECT 
    COUNT(*) AS total_customers,
    SUM(Personal_Loan) AS loan_customers,
    ROUND(AVG(Personal_Loan) * 100, 2) AS loan_acceptance_rate_pct
FROM loan_customers;
"""

pd.read_sql(query_1, conn)

,total_customers,loan_customers,loan_acceptance_rate_pct
0,5000,480,9.6


In [10]:
### Business Interpretation

- The overall personal loan acceptance rate is 9.6%, indicating conservative credit exposure across the retail customer portfolio.
- This suggests selective lending practices or targeted marketing of personal loans.
- Given the low penetration, overall portfolio risk is contained, but credit exposure is likely concentrated within specific customer segments, warranting further segmentation analysis.


SyntaxError: invalid syntax (671793346.py, line 3)

In [6]:
#Loan acceptance by income segment
query_2 = """
SELECT 
    CASE 
        WHEN Income < 50 THEN 'Low Income'
        WHEN Income BETWEEN 50 AND 100 THEN 'Middle Income'
        ELSE 'High Income'
    END AS income_segment,
    COUNT(*) AS customers,
    ROUND(AVG(Personal_Loan) * 100, 2) AS acceptance_rate_pct
FROM loan_customers
GROUP BY income_segment
ORDER BY acceptance_rate_pct DESC;
"""

pd.read_sql(query_2, conn)

,income_segment,customers,acceptance_rate_pct
0,High Income,1212,36.14
1,Middle Income,1919,2.19
2,Low Income,1869,0.00


In [ ]:
### Business Interpretation

- Loan acceptance is heavily skewed toward high-income customers, with a 36.1% acceptance rate, while middle-income and low-income segments show minimal to zero uptake.
- This highlights income as the primary determinant of personal loan exposure and reflects conservative underwriting practices.
- Although this strategy limits default risk, it also results in exposure concentration within the high-income segment, increasing portfolio concentration risk.
- The middle-income segment may represent a controlled growth opportunity if supported by stricter credit screening and risk-based pricing.

In [5]:
#Loan acceptance by education level
query_3 = """
SELECT 
    Education,
    COUNT(*) AS customers,
    ROUND(AVG(Personal_Loan) * 100, 2) AS acceptance_rate_pct
FROM loan_customers
GROUP BY Education
ORDER BY acceptance_rate_pct DESC;
"""

pd.read_sql(query_3, conn)

,Education,customers,acceptance_rate_pct
0,3,1501,13.66
1,2,1403,12.97
2,1,2096,4.44


In [ ]:
### Business Interpretation

- Loan acceptance rates increase materially with education level, with graduate and professionally educated customers showing acceptance rates close to 13–14%.
- Education acts as a proxy for income stability and creditworthiness, aligning with conservative retail banking underwriting practices.
- Despite forming the largest customer base, undergraduate customers exhibit significantly lower loan acceptance, limiting exposure to potentially higher-risk profiles.
- Combined with income analysis, education reinforces the concentration of credit exposure among financially stable customer segments.

In [7]:
#High exposure customer count
query_4 = """
SELECT 
    COUNT(*) AS high_exposure_customers
FROM loan_customers
WHERE Income > 100 AND Personal_Loan = 1;
"""

pd.read_sql(query_4, conn)

,high_exposure_customers
0,438


In [ ]:
### Business Interpretation

- A total of 438 customers fall into the high-income and loan-accepted category, accounting for more than 90% of all personal loan customers.
- This indicates that personal loan exposure is highly concentrated within the high-income segment, reflecting a conservative credit risk strategy.
- While this approach limits default risk, it introduces concentration risk, making the portfolio sensitive to adverse events affecting a specific customer segment.
- Diversifying exposure across additional low-risk segments could improve portfolio balance without significantly increasing credit risk.


In [8]:
conn.close()